<a href="https://colab.research.google.com/github/tjhung23/MBC/blob/main/01_%C4%90%E1%BB%91i_So%C3%A1t_C%E1%BB%95ng_thanh_to%C3%A1n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 10.1 MB/s eta 0:00:00


In [ ]:
# lấy báo cáo đối soát cổng thanh toán
import glob
import os
import pandas as pd
import numpy as np
import csv
from datetime import datetime
import getpass
import pytz
import time
# --------------------------------------------------------------------------------------------------------------------------
# Lấy thời gian bắt đầu
start_time = time.time()

vn_tz = pytz.timezone('Asia/Ho_Chi_Minh')
now = datetime.now(vn_tz)

# Tạo timestamp
timestamp = now.strftime("%Y-%m-%d_%H-%M")
user = getpass.getuser()

#---------------------------------------------------------------------------------------------------------------------------
xlpath = '/content/drive/My Drive/KeToan_HachToanGiaoDich/CTT/2025/T06/'
result = glob.glob(xlpath + "*.*")
data = pd.DataFrame(columns=['Date', 'Ref', 'Amount', 'Bank'])
cuoc_data = 4500
cuoc_thoai = 500
cuoc_sms = 300
BIDV_list = []
ONEPAY_list = []
VNPAY_list = []
TCB_list = []
VNPOST_list = []
VTPOST_list = []
dwh_list = []
dwh_bh_list = []
#---------------------------------------------------------------------------------------------------------------------------
def generate_note(sim_type):
    if sim_type in ['e sim', 'esim']:
        return '338711 - Esim'
    elif sim_type == 'Sim vật lý':
        return '338711 - Sim vật lý'
    else:
        return 'Khác'
#---------------------------------------------------------------------------------------------------------------------------
for filename in result:
  # try:
    if filename.endswith((".xlsx", ".xls")):
      df = pd.read_excel(filename, dtype=str)
#---------------------------------------------------------------------------------------------------------------------------
      #DWH_doi soat CTT

      if df.columns[0] == 'Mã giao dịch':
        WH = df.copy()
        dwh = df.copy()
        dwh = dwh[dwh['Trạng thái thanh toán'] == 'Thành công']
        dwh = dwh[dwh['Cổng thanh toán']!= "MAIN_ACCOUNT"]

        cols = ['Giá trị đơn', 'Giảm giá', 'Phí thanh toán', 'Tổng tiền', 'Phí cổng thanh toán', 'Giá trị còn lại', 'SL data', 'SL SMS', 'SL Voice', 'Giá Data', 'Giá SMS', 'Giá Voice', 'Giá gói'] ## cột cần cast sang int
        dwh[cols] = dwh[cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
        # Tính giá mới cho đơn hàng, Logic của Kế toán
        dwh['Giá SMS2'] = dwh['SL SMS'] * cuoc_sms * ((dwh['Loại đơn hàng'] == 'Mua gói cước') & (dwh['Gói cước'] == 'TỰ DO TRẢI NGHIỆM')).astype(int)

        dwh['Giá Thoại'] = dwh.apply(lambda row: row['SL Voice'] * cuoc_thoai if row['Loại đơn hàng'] == 'Mua gói cước' and row['Gói cước'] == 'TỰ DO TRẢI NGHIỆM' else row['Giá trị đơn'] if row['Loại đơn hàng'] == 'Mua gói cước' and str(row['Gói cước']).startswith('V') else 0, axis=1)

        dwh['Giá Data2'] = dwh.apply(lambda row: row['Tổng tiền'] - row['Giá SMS2'] - row['Giá Thoại'] if row['Loại đơn hàng'] == 'Mua gói cước' and row['Gói cước'] == 'TỰ DO TRẢI NGHIỆM' else row['Tổng tiền'] if row['Loại đơn hàng'] == 'Mua gói cước' and str(row['Gói cước']).startswith('D') else row['Tổng tiền'] if row['Loại đơn hàng'] == 'VOUCHER_DATA' else 0, axis=1)

        dwh['Giá combo'] = dwh.apply(lambda row: row['Giá trị đơn'] - row['Giá Thoại'] - row['Giá Data2'] - row['Giá SMS2'] if pd.notnull(row['Giá trị đơn']) and pd.notnull(row['Giá Thoại']) and pd.notnull(row['Giá Data2']) and pd.notnull(row['Giá SMS2']) else 0, axis=1)
        dwh['Mã giao dịch'] = dwh['Mã giao dịch'].astype(str).str.strip()
        dwh_list.append(dwh) # Bỏ vào Rổ ONEPAY
        # # Create SDT_DAT
        # SDT_DAT = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['SĐT đặt']

        # # Map SDT_DAT to 'SDT đặt' => lấy SDT từ bên báo cáo D2C qua, vì báo cáo CTT là SDT khác witel
        # dwh['SDT đặt'] = dwh['Mã đơn hàng'].map(SDT_DAT)

        # # Create KYC
        # KYC = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Đã KYC']

        # # Map 'KYC' => để biết thuê bao mua có KYC chưa
        # dwh['KYC'] = dwh['Mã đơn hàng'].map(KYC)


        # # Create 'new' column based on non-NaN values in 'SDT đặt' after mapping
        # dwh['new'] = dwh['SDT đặt'].notna().map({True: 'new', False: ''})

        # # Apply fillna operations and convert to string
        # dwh['SDT đặt'] = dwh['SDT đặt'].fillna(dwh['SĐT khách hàng']).fillna(0).astype(str)

        # #Discount => lấy value discount nếu có
        # giam_gia = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Giảm giá']
        # dwh['Giảm giá'] = dwh['Mã đơn hàng'].map(giam_gia).fillna(0).astype(int)

#---------------------------------------------------------------------------------------------------------------------------
      #DWH_BH => báo cáo bán hàng D2C

      if df.columns[0] == 'Mã vận đơn MBC':
        dwh_bh = df.copy()
        # các cột cần cast sang int để tính
        cols = ['Giá trị đơn hàng', 'Giảm giá','Phí ship', 'Tổng tiền(đã chia)', 'Tổng tiền(chưa chia)', 'Giá bộ KIT sim trắng cước hòa mạng','Giá call',
                'Giá gói','Đã KYC','Tổng data','Giá data','Giảm giá data','Tổng tiền call','Tổng call','Tổng tiền data','Giảm giá call','Tổng SMS','Giá SMS','Giảm giá SMS','Tổng tiền SMS']
        dwh_bh[cols] = dwh_bh[cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
        # Tính giá mới cho đơn hàng
        dwh_bh['Giá data2'] = dwh_bh['Tổng data'] * cuoc_data * (dwh_bh['Tên gói'] == "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Giá thoại2'] = (dwh_bh['Tổng call']- dwh_bh['Giảm giá call']) * cuoc_thoai * (dwh_bh['Tên gói'] == "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Giá SMS2'] = (dwh_bh['Tổng SMS'] - dwh_bh['Giảm giá SMS']) * cuoc_sms * (dwh_bh['Tên gói'] == "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Chênh lệch'] = (dwh_bh['Giá gói'] - (dwh_bh['Giá data2'] + dwh_bh['Giá thoại2'] + dwh_bh['Giá SMS2'])) * (dwh_bh['Tên gói'] == "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Combo - 1 tháng'] = dwh_bh['Giá gói'] * (dwh_bh['Tên gói']!= "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Giá hòa mạng'] = np.where(dwh_bh['Giá bộ KIT sim trắng cước hòa mạng'] == 0, 0,np.where(dwh_bh['Giá bộ KIT sim trắng cước hòa mạng'] == 20000, 20000, 25000))
        dwh_bh['Giá sim'] = dwh_bh['Giá bộ KIT sim trắng cước hòa mạng'] - dwh_bh['Giá hòa mạng']
        dwh_bh_list.append(dwh_bh) # bỏ vào rổ


In [ ]:
# Tạo một Series ánh xạ từ dwh_bh
map_muasim = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Tên gói']

# Chỉ cập nhật nếu là "Mua sim" và cột Gói cước đang trống
mask_muasim = (dwh['Loại đơn hàng'] == 'Mua sim') & (dwh['Gói cước'].isna())

# Gán giá trị mới từ mapping
dwh.loc[mask_muasim, 'Gói cước'] = dwh.loc[mask_muasim, 'Mã đơn hàng'].map(map_muasim)


In [ ]:
        # Create SDT_DAT
        SDT_DAT = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['SĐT đặt']

        # Map SDT_DAT to 'SDT đặt' => lấy SDT từ bên báo cáo D2C qua, vì báo cáo CTT là SDT khác witel
        dwh['SDT đặt'] = dwh['Mã đơn hàng'].map(SDT_DAT)

        # Create KYC
        KYC = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Đã KYC']

        # Map 'KYC' => để biết thuê bao mua có KYC chưa
        dwh['KYC'] = dwh['Mã đơn hàng'].map(KYC)


        # Create 'new' column based on non-NaN values in 'SDT đặt' after mapping
        dwh['new'] = dwh['SDT đặt'].notna().map({True: 'new', False: ''})

        # Apply fillna operations and convert to string
        dwh['SDT đặt'] = dwh['SDT đặt'].fillna(dwh['SĐT khách hàng']).fillna(0).astype(str)

        #Discount => lấy value discount nếu có
        giam_gia = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Giảm giá']
        dwh['Giảm giá'] = dwh['Mã đơn hàng'].map(giam_gia).fillna(0).astype(int)

if dwh_list:
    dwh = pd.concat(dwh_list, ignore_index=True)
    dwh = dwh.drop_duplicates(keep='first')  # Loại trùng toàn bộ dòng

if dwh_bh_list:
    dwh_bh = pd.concat(dwh_bh_list, ignore_index=True)
    dwh_bh = dwh_bh.drop_duplicates(keep='first')  # Loại trùng toàn bộ dòng

In [ ]:
# đặt map SDT, KYC, Giảm giá ngòi loop để code nhanh hơn
SDT= dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['SDT đặt']
KYC_check = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['KYC']
giam_gia = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giảm giá']
# Create SDT_DAT
SDT_DAT = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['SĐT đặt']

# Map SDT_DAT to 'SDT đặt' => lấy SDT từ bên báo cáo D2C qua, vì báo cáo CTT là SDT khác witel
dwh['SDT đặt'] = dwh['Mã đơn hàng'].map(SDT_DAT)

In [ ]:
### load cổng thanh toán và map đơn hàng
result_bank = glob.glob(xlpath + "*.*")
#---------------------------------------------------------------------------------------------------------------------------
for filename in result_bank:
  try:
    if filename.endswith((".xlsx", ".xls")):
      df = pd.read_excel(filename, dtype=str)

# #---------------------------------------------------------------------------------------------------------------------------
#       # BIDV => ELT cho sổ phụ BIDV
      if df.shape[0] > 8 and df.shape[1] > 8 and df.iloc[0, 3] == "NGÂN HÀNG TMCP ĐẦU TƯ VÀ PHÁT TRIỂN VIỆT NAM": # nhận dạng BIDV
        tmp_bidv = df.copy()
        tmp = df.copy()
        tmp_bidv = df.iloc[8:,1:].copy()
        tmp_bidv.columns = tmp_bidv.iloc[0]
        tmp_bidv = tmp_bidv.iloc[2:].copy().reset_index(drop=True)
        tmp_bidv = tmp_bidv.loc[:, tmp_bidv.columns.notna()]
        tmp_bidv = tmp_bidv.copy()
        tmp_bidv.loc[:, 'Ngày hạch toán'] = tmp_bidv['Thời gian hạch toán'].str.split(' ').str[0].str.replace('/', '-', regex=False)
        tmp_bidv.loc[:, 'Mã Hóa đơn'] = tmp_bidv['Mã Hóa đơn'].astype(str).str.strip()
        tmp_bidv['Mã đơn hàng'] = tmp_bidv['Mã Hóa đơn'].map(dwh.drop_duplicates('Mã giao dịch').set_index('Mã giao dịch')['Mã đơn hàng']).fillna('Không có trong DWH')
        tmp_bidv['Loại đơn hàng'] = tmp_bidv['Mã Hóa đơn'].map(dwh.drop_duplicates('Mã giao dịch').set_index('Mã giao dịch')['Loại đơn hàng']).fillna('Không có trong DWH')
        tmp_bidv['Số tiền thu hộ'] = pd.to_numeric(tmp_bidv['Số tiền thu hộ'], errors='coerce').fillna(0).astype(int)
        tmp_bidv['Đơn vị'] = 'SALES - Online'
        tmp_bidv['SDT'] = tmp_bidv['Mã đơn hàng'].map(SDT)
        tmp_bidv['KYC'] = tmp_bidv['Mã đơn hàng'].map(KYC_check)
        tmp_bidv['Giảm giá'] = tmp_bidv['Mã đơn hàng'].map(giam_gia)
        tmp_bidv = tmp_bidv[tmp_bidv['Ngày hạch toán'].notna()]
        BIDV_list.append(tmp_bidv) # bỏ vào rổ BIDV


#---------------------------------------------------------------------------------------------------------------------------
      # VNPT_EPAY => ETL cho file giao dịch EPAY
      elif df.shape[0] > 4 and df.shape[1] > 6 and df.iloc[2, 1] == "MID": # nhận dạng EPAY
        tmp_vnpay = df.copy()
        tmp_vnpay = df.iloc[2:, :].copy()
        tmp_vnpay.columns = tmp_vnpay.iloc[0]
        tmp_vnpay = tmp_vnpay[2:].reset_index(drop=True)
        # Cast các cột sang int để tính
        cols = ['Giá trị giao dịch','Giá trị tạm giữ', 'Phí xử lý giao dịch', 'Phí thanh toán', 'Phí lệch với Domestic', 'Phí trả góp trên giá trị trả góp (tổng cộng)',
               'Tỷ lệ phí trả góp enduser chịu','Phí trả góp merchant chịu', 'Phí hủy chuyển đổi', 'Tổng cộng']
        # Làm sạch tên cột
        tmp_vnpay.columns = tmp_vnpay.columns.astype(str).str.strip()

        # Lọc lại chỉ những cột tồn tại
        existing_cols = [col for col in cols if col in tmp_vnpay.columns]

        # Chuyển đổi chỉ những cột đó
        tmp_vnpay[existing_cols] = tmp_vnpay[existing_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
        tmp_vnpay['SDT'] = tmp_vnpay['Mã đặt hàng'].map(SDT)

        tmp_vnpay['KYC'] = tmp_vnpay['Mã đặt hàng'].map(KYC_check)

        tmp_vnpay['Giảm giá'] = tmp_vnpay['Mã đặt hàng'].map(giam_gia)

        loai_from_dwh = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Loại đơn hàng']

        # map giá trị, chưa có DWH thì ra NaN
        mapped = tmp_vnpay['Mã đặt hàng'].map(loai_from_dwh)

        # nếu mapped là NaN, tức không có trong DWH, gán nhãn "Không có trong DWH"
        tmp_vnpay['Loại đơn hàng'] = np.where(mapped.isna(),'Không có trong DWH',mapped)

        loai_from_goi = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Gói cước']
        map_goicuoc = tmp_vnpay['Mã đặt hàng'].map(loai_from_goi)
        tmp_vnpay['Gói cước'] = np.where(map_goicuoc.isna(),' ',map_goicuoc)
        # tmp_vnpay['Gói cước'] = tmp_vnpay['Gói cước'].replace(['', ' ', 'nan'], np.nan)
        tmp_vnpay.loc[(tmp_vnpay['Gói cước'].astype(str).str.strip().isin(['', ' ', 'nan'])) & (tmp_vnpay['Loại đơn hàng'] == 'VOUCHER_DATA'),'Gói cước'] = 'VOUCHER_DATA'

        tmp_vnpay['Loại sim'] = np.where(tmp_vnpay['Loại đơn hàng'] == 'Mua sim',tmp_vnpay['Mã đặt hàng'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Loại sim']).fillna(0),
                              np.where(tmp_vnpay['Loại đơn hàng'] == 'Đổi sim',tmp_vnpay['Mã đặt hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Loại sim']).fillna(0),0))

        tmp_vnpay_affiliate = tmp_vnpay['Mã đặt hàng'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Affiliate partner'])
        tmp_vnpay['Đơn vị'] = np.where(tmp_vnpay_affiliate == 'ACCESSTRADE','SALES - Online - ACT','SALES - Online')
        tmp_vnpay['Tổng tiền thu'] = tmp_vnpay['Giá trị giao dịch']

        so_luong_sim = dwh_bh['Mã vận đơn MBC'].value_counts()
        tmp_vnpay['Số lượng sim'] = tmp_vnpay['Mã đặt hàng'].map(so_luong_sim).fillna(0).astype(int)

        # 1. Chọn các dòng có Phí hòa mạng > 0
        mask_hoa_mang = dwh_bh['Giá hòa mạng'] > 0

        # 2. Đếm số lần mỗi Mã vận đơn MBC phát sinh Phí hòa mạng
        so_luong_hoa_mang = (dwh_bh.loc[mask_hoa_mang, 'Mã vận đơn MBC'].value_counts())

        # 3. Map vào tmp_vnpay_MISA
        tmp_vnpay['Số lượng hòa mạng'] = (tmp_vnpay['Mã đặt hàng'].map(so_luong_hoa_mang).fillna(0).astype(int))



        tong_gia_sim = dwh_bh.groupby('Mã vận đơn MBC')['Giá sim'].sum()
        tmp_vnpay['Tiền sim trắng'] = np.where(tmp_vnpay['Loại đơn hàng'] == "Đổi sim",5000,tmp_vnpay['Mã đặt hàng'].map(tong_gia_sim).fillna(0))

        tong_gia_hoa_mang = dwh_bh.groupby('Mã vận đơn MBC')['Giá hòa mạng'].sum()
        tmp_vnpay['Phí hòa mạng'] = np.where(tmp_vnpay['Loại đơn hàng'] == "Mua sim",tmp_vnpay['Mã đặt hàng'].map(tong_gia_hoa_mang).fillna(0),0)

        tong_gia_thoai = dwh_bh.groupby('Mã vận đơn MBC')['Giá thoại2'].sum()
        tmp_vnpay['Tiền gói - Thoại'] = np.where(tmp_vnpay['Loại đơn hàng'] == "Mua gói cước", tmp_vnpay['Mã đặt hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Thoại']).fillna(0).astype(int),
                                    np.where(tmp_vnpay['Loại đơn hàng'] == "Mua sim",tmp_vnpay['Mã đặt hàng'].map(tong_gia_thoai).fillna(0),0))

        tong_gia_sms = dwh_bh.groupby('Mã vận đơn MBC')['Giá SMS2'].sum()
        tmp_vnpay['Tiền gói - SMS'] = np.where(tmp_vnpay['Loại đơn hàng'] == "Mua gói cước", tmp_vnpay['Mã đặt hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá SMS2']).fillna(0).astype(int),
                                    np.where(tmp_vnpay['Loại đơn hàng'] == "Mua sim",tmp_vnpay['Mã đặt hàng'].map(tong_gia_sms).fillna(0),0))

        tong_gia_data = dwh_bh.groupby('Mã vận đơn MBC')['Giá data2'].sum()

        tmp_vnpay['Tiền gói - Data'] = np.where(tmp_vnpay['Loại đơn hàng'] == "Mua gói cước",tmp_vnpay['Mã đặt hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Data2']).fillna(0).astype(int),
                                    np.where(tmp_vnpay['Loại đơn hàng'] == "Mua sim",tmp_vnpay['Mã đặt hàng'].map(tong_gia_data).fillna(0),
                                    np.where(tmp_vnpay['Loại đơn hàng'] == "VOUCHER_DATA",tmp_vnpay['Tổng tiền thu'].fillna(0),0)))

        tong_gia_combo = dwh_bh.groupby('Mã vận đơn MBC')['Combo - 1 tháng'].sum()
        tmp_vnpay['Tiền gói - Combo 1 tháng'] = np.where(tmp_vnpay['Loại đơn hàng'] == "Mua gói cước", tmp_vnpay['Mã đặt hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá combo']).fillna(0).astype(int),
                                    np.where(tmp_vnpay['Loại đơn hàng'] == "Mua sim",tmp_vnpay['Mã đặt hàng'].map(tong_gia_combo).fillna(0),0))

        tmp_vnpay['Doanh thu khác'] = np.where(tmp_vnpay['Loại đơn hàng'] == "Đổi sim",20000,np.where(tmp_vnpay['Loại đơn hàng'] == "Khôi phục số",tmp_vnpay['Tổng tiền thu'],0))

        tmp_vnpay['Tiền ship'] = np.where(tmp_vnpay['Loại đơn hàng'] == "Đổi sim",tmp_vnpay['Tổng tiền thu'] - tmp_vnpay['Tiền sim trắng'] - tmp_vnpay['Doanh thu khác'],
                                    np.where(tmp_vnpay['Loại đơn hàng'] == "Mua sim",tmp_vnpay['Mã đặt hàng'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Phí ship']).fillna(0),0))

        tmp_vnpay['Topup'] = np.where(tmp_vnpay['Loại đơn hàng'] == "Topup",tmp_vnpay['Tổng tiền thu'],0)

        tmp_vnpay['Voucher'] = np.where(tmp_vnpay['Loại đơn hàng'] == "VOUCHER",tmp_vnpay['Tổng tiền thu'],0)

        cols_to_subtract = ['Tiền sim trắng', 'Phí hòa mạng', 'Tiền gói - Thoại', 'Tiền gói - SMS',    'Tiền gói - Data', 'Tiền gói - Combo 1 tháng', 'Doanh thu khác','Tiền ship', 'Topup', 'Voucher']
        tmp_vnpay['Check'] = tmp_vnpay['Tổng tiền thu'] - tmp_vnpay[cols_to_subtract].sum(axis=1)
        tmp_vnpay['Ngày phát sinh giao dịch'] = pd.to_datetime(tmp_vnpay['Ngày phát sinh giao dịch'], errors='coerce', dayfirst=True)
        tmp_vnpay['Ngày giao dịch'] = tmp_vnpay['Ngày phát sinh giao dịch'].dt.strftime('%d-%m-%Y')
        VNPAY_list.append(tmp_vnpay) # bỏ vào rổ VNPAY


#---------------------------------------------------------------------------------------------------------------------------
      # Onepay
      elif df.shape[0] > 6 and df.iloc[2, 0] == "TÌM KIẾM GIAO DỊCH": # nhận dạng file giao dịch ONEPAY
        # print(filename)
        tmp_onepay = df.copy()
        tmp_onepay = df.iloc[5:, :].copy()
        tmp_onepay.columns = tmp_onepay.iloc[0]
        tmp_onepay = tmp_onepay[1:].reset_index(drop=True)
        tmp_onepay.columns = tmp_onepay.columns.str.strip()
        tmp_onepay = tmp_onepay[tmp_onepay['Trạng thái giao dịch'] == 'Thành công']
        tmp_onepay['Ngày giao dịch'] =tmp_onepay['Ngày'].astype(str).str.split(" ").str[0]
        tmp_onepay[['Year', 'Month', 'Day']] = tmp_onepay['Ngày giao dịch'].str.split('-', expand=True)
        tmp_onepay['Ngày giao dịch'] = tmp_onepay['Day'] + '-' + tmp_onepay['Month'] + '-' + tmp_onepay['Year']
        tmp_onepay = tmp_onepay.drop(['Year', 'Month', 'Day'], axis=1)
        tmp_onepay['Số tiền'] = pd.to_numeric(tmp_onepay['Số tiền'], errors='coerce').fillna(0).astype(int)
        tmp_onepay['SDT'] = tmp_onepay['Mã đơn hàng'].map(SDT)

        tmp_onepay['KYC'] = tmp_onepay['Mã đơn hàng'].map(KYC_check)

        tmp_onepay['Giảm giá'] = tmp_onepay['Mã đơn hàng'].map(giam_gia)

        loai_from_dwh = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Loại đơn hàng']
        # # map giá trị, chưa có DWH thì ra NaN
        loaidh_mapped = tmp_onepay['Mã đơn hàng'].map(loai_from_dwh)

        # # nếu mapped là NaN, tức không có trong DWH, gán nhãn "Không có trong DWH"
        tmp_onepay['Loại đơn hàng'] = np.where(loaidh_mapped.isna(),'Không có trong DWH',loaidh_mapped)

        loai_from_goi = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Gói cước']
        map_goicuoc = tmp_onepay['Mã đơn hàng'].map(loai_from_goi)
        tmp_onepay['Gói cước'] = np.where(map_goicuoc.isna(),' ',map_goicuoc)
        # tmp_onepay['Gói cước'] = tmp_onepay['Gói cước'].replace(['', ' ', 'nan'], np.nan)
        tmp_onepay.loc[(tmp_onepay['Gói cước'].astype(str).str.strip().isin(['', ' ', 'nan'])) & (tmp_onepay['Loại đơn hàng'] == 'VOUCHER_DATA'),'Gói cước'] = 'VOUCHER_DATA'

        tmp_onepay['Loại sim'] = np.where(tmp_onepay['Loại đơn hàng'] == 'Mua sim',tmp_onepay['Mã đơn hàng'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Loại sim']).fillna(0),
                              np.where(tmp_onepay['Loại đơn hàng'] == 'Đổi sim',tmp_onepay['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Loại sim']).fillna(0),0))

        tmp_onepay_affiliate = tmp_onepay['Mã đơn hàng'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Affiliate partner'])
        tmp_onepay['Đơn vị'] = np.where(tmp_onepay_affiliate == 'ACCESSTRADE','SALES - Online - ACT','SALES - Online')
        tmp_onepay['Tổng tiền thu'] = tmp_onepay['Số tiền']

        def tinh_phi(row): # Hàm Tính Phí giao dịch của ONEEPAY
          kenh = row.get('Loại thẻ/ Tên ngân hàng', '').strip()
          hinh_thuc = row.get('Kênh thanh toán', '').strip()

          # Đưa về chữ thường để so sánh chính xác
          kenh = kenh.lower()
          hinh_thuc = hinh_thuc.lower()

          # Xử lý số tiền
          try:
              so_tien = float(str(row['Tổng tiền thu']).replace(',', '').strip())
          except (ValueError, TypeError):
              return 0

          # Ưu tiên xử lý theo 'kenh'
          if kenh == "zalopay":
              return 0.01 * so_tien
          elif kenh == "momo":
              return 0.0175 * so_tien
          elif kenh == "moca":
              return 0.0088 * so_tien
          elif kenh == "shopeepay":
              return 0.01 * so_tien
          elif kenh == "viettelpay":
              return 0.0088 * so_tien
          elif kenh == "visa":
              return 2200 + (0.018 * so_tien)
          elif kenh == "vnpayqr":
              return 0.0088 * so_tien
          elif kenh == "mastercard":
              return 2200 + (0.018 * so_tien)
          elif kenh == "jcb":
              return 2200 + (0.018 * so_tien)
          elif kenh == "amex":
              return 2200 + (0.0275 * so_tien)
          elif kenh == "atm card":
              return 0.0088 * so_tien

          # Nếu 'kenh' không match, xét 'hinh_thuc'
          elif hinh_thuc == "qr":
              return 0.0088 * so_tien

          return 0

        # Áp dụng vào DataFrame
        tmp_onepay['Phí'] = tmp_onepay.apply(tinh_phi, axis=1)
        tmp_onepay['Tiền thu'] = tmp_onepay['Tổng tiền thu'] - tmp_onepay['Phí']


        so_luong_sim = dwh_bh['Mã vận đơn MBC'].value_counts()
        tmp_onepay['Số lượng sim'] = tmp_onepay['Mã đơn hàng'].map(so_luong_sim).fillna(0).astype(int)

        tong_gia_sim = dwh_bh.groupby('Mã vận đơn MBC')['Giá sim'].sum()
        tmp_onepay['Tiền sim trắng'] = np.where(tmp_onepay['Loại đơn hàng'] == "Đổi sim",5000,
                                             np.where(tmp_onepay['Loại đơn hàng'] == "Mua sim",tmp_onepay['Mã đơn hàng'].map(tong_gia_sim).fillna(0),0))

        tong_gia_hoa_mang = dwh_bh.groupby('Mã vận đơn MBC')['Giá hòa mạng'].sum()
        tmp_onepay['Phí hòa mạng'] = np.where(tmp_onepay['Loại đơn hàng'] == "Mua sim",tmp_onepay['Mã đơn hàng'].map(tong_gia_hoa_mang).fillna(0),0)

        # 1. Chọn các dòng có Phí hòa mạng > 0
        mask_hoa_mang = dwh_bh['Giá hòa mạng'] > 0

        # 2. Đếm số lần mỗi Mã vận đơn MBC phát sinh Phí hòa mạng
        so_luong_hoa_mang = (dwh_bh.loc[mask_hoa_mang, 'Mã vận đơn MBC'].value_counts())

        # 3. Map vào EPAY_MISA
        tmp_onepay['Số lượng hòa mạng'] = (tmp_onepay['Mã đơn hàng'].map(so_luong_hoa_mang).fillna(0).astype(int))

        tong_gia_thoai = dwh_bh.groupby('Mã vận đơn MBC')['Giá thoại2'].sum()
        tmp_onepay['Tiền gói - Thoại'] = np.where(tmp_onepay['Loại đơn hàng'] == "Mua gói cước", tmp_onepay['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Thoại']).fillna(0).astype(int),
                                    np.where(tmp_onepay['Loại đơn hàng'] == "Mua sim",tmp_onepay['Mã đơn hàng'].map(tong_gia_thoai).fillna(0),0))

        tong_gia_sms = dwh_bh.groupby('Mã vận đơn MBC')['Giá SMS2'].sum()
        tmp_onepay['Tiền gói - SMS'] = np.where(tmp_onepay['Loại đơn hàng'] == "Mua gói cước", tmp_onepay['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá SMS2']).fillna(0).astype(int),
                                    np.where(tmp_onepay['Loại đơn hàng'] == "Mua sim",tmp_onepay['Mã đơn hàng'].map(tong_gia_sms).fillna(0),0))

        tong_gia_data = dwh_bh.groupby('Mã vận đơn MBC')['Giá data2'].sum()
        tmp_onepay['Tiền gói - Data'] = np.where(tmp_onepay['Loại đơn hàng'] == "Mua gói cước",tmp_onepay['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Data2']).fillna(0).astype(int),
                            np.where(tmp_onepay['Loại đơn hàng'] == "Mua sim",tmp_onepay['Mã đơn hàng'].map(tong_gia_data).fillna(0),
                            np.where(tmp_onepay['Loại đơn hàng'] == "VOUCHER_DATA",tmp_onepay['Tổng tiền thu'].fillna(0),0)))


        tong_gia_combo = dwh_bh.groupby('Mã vận đơn MBC')['Combo - 1 tháng'].sum()
        tmp_onepay['Tiền gói - Combo 1 tháng'] = np.where(tmp_onepay['Loại đơn hàng'] == "Mua gói cước", tmp_onepay['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá combo']).fillna(0).astype(int),
                                                              np.where(tmp_onepay['Loại đơn hàng'] == "Mua sim",tmp_onepay['Mã đơn hàng'].map(tong_gia_combo).fillna(0),0))

        tong_gia_thoai = dwh_bh.groupby('Mã vận đơn MBC')['Giá thoại2'].sum()
        tmp_onepay['Tiền gói - Thoại'] = np.where(tmp_onepay['Loại đơn hàng'] == "Mua gói cước", tmp_onepay['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Thoại']).fillna(0).astype(int),
                                    np.where(tmp_onepay['Loại đơn hàng'] == "Mua sim",tmp_onepay['Mã đơn hàng'].map(tong_gia_thoai).fillna(0),0))

        tmp_onepay['Doanh thu khác'] = np.where(tmp_onepay['Loại đơn hàng'] == "Đổi sim",20000,np.where(tmp_onepay['Loại đơn hàng'] == "Khôi phục số",tmp_onepay['Tổng tiền thu'],0))

        tmp_onepay['Tiền ship'] = np.where(tmp_onepay['Loại đơn hàng'] == "Đổi sim",tmp_onepay['Tổng tiền thu'] - tmp_onepay['Tiền sim trắng'] - tmp_onepay['Doanh thu khác'],
                                    np.where(tmp_onepay['Loại đơn hàng'] == "Mua sim",tmp_onepay['Mã đơn hàng'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Phí ship']).fillna(0),0))

        tmp_onepay['Topup'] = np.where(tmp_onepay['Loại đơn hàng'] == "Topup",tmp_onepay['Tổng tiền thu'],0)

        tmp_onepay['Voucher'] = np.where(tmp_onepay['Loại đơn hàng'] == "VOUCHER",tmp_onepay['Tổng tiền thu'],0)

        cols_to_subtract = ['Tiền sim trắng', 'Phí hòa mạng', 'Tiền gói - Thoại', 'Tiền gói - SMS','Tiền gói - Combo 1 tháng'  ,  'Tiền gói - Data',  'Doanh thu khác','Tiền ship', 'Topup', 'Voucher']
        tmp_onepay['Check'] = tmp_onepay['Tổng tiền thu'] - tmp_onepay[cols_to_subtract].sum(axis=1)
        ONEPAY_list.append(tmp_onepay) # Bỏ vào Rổ ONEPAY
# #---------------------------------------------------------------------------------------------------------------------------

# #---------------------------------------------------------------------------------------------------------------------------
    # CSV (TCB) file TCB dạng CSV
    elif filename.endswith(".csv"):
      rows = []
      with open(filename, newline='', encoding='utf-8') as csvfile:
          # print(filename)
          reader = csv.reader(csvfile)
          for row in reader:
              rows.append(row)

# #---------------------------------------------------------------------------------------------------------------------------
      if len(rows) > 6:

        df_csv = pd.DataFrame(rows)
        if df_csv.shape[1] >= 7:
          TCB = df_csv.copy()
          tmp_tcb = df_csv.iloc[9:, :7].reset_index(drop=True)
          tmp_tcb.columns = tmp_tcb.iloc[0]
          tmp_tcb = tmp_tcb[1:]
          tmp_tcb = tmp_tcb.drop(columns=['No/Debit'])
          tmp_tcb['Co/Credit'] = tmp_tcb['Co/Credit'].str.replace(',', '').str.strip()
          tmp_tcb['Co/Credit'] = pd.to_numeric(tmp_tcb['Co/Credit'], errors='coerce').fillna(0).astype('Int64')
          tmp_tcb = tmp_tcb[tmp_tcb['Co/Credit'] != 0].reset_index(drop=True)
          tmp_tcb['So but toan/Txn No'] = tmp_tcb['So but toan/Txn No'].astype(str).str.split("\\").str[0]
          tmp_tcb['Ngay hach toan/Booking Date'] = tmp_tcb['Ngay hach toan/Booking Date'].str.replace('/', '-', regex=False)
          # tmp_tcb["Ngay hach toan/Booking Date"] = tmp_tcb["Ngay hach toan/Booking Date"].apply(lambda x: datetime.strptime(x, "%d-%m-%y").strftime("%d-%m-%Y"))
          tmp_tcb['Mã đơn hàng'] = tmp_tcb['So but toan/Txn No'].map(dwh.drop_duplicates('Mã giao dịch').set_index('Mã giao dịch')['Mã đơn hàng']).fillna('Không có trong DWH')
          loai_from_dwh = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Loại đơn hàng']
          tmp_tcb['Tổng tiền thu'] = tmp_tcb['Co/Credit']
          tmp_tcb['SDT'] = tmp_tcb['Mã đơn hàng'].map(SDT)
          tmp_tcb['KYC'] = tmp_tcb['Mã đơn hàng'].map(KYC_check)
          tmp_tcb['Giảm giá'] = tmp_tcb['Mã đơn hàng'].map(giam_gia)
        # # map giá trị, chưa có DWH thì ra NaN
          loaidh_mapped = tmp_tcb['Mã đơn hàng'].map(loai_from_dwh)

        # # nếu mapped là NaN, tức không có trong DWH, gán nhãn "Không có trong DWH"
          tmp_tcb['Loại đơn hàng'] = np.where(loaidh_mapped.isna(),'Không có trong DWH',loaidh_mapped)


          loai_from_goi = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Gói cước']
          map_goicuoc = tmp_tcb['Mã đơn hàng'].map(loai_from_goi)
          tmp_tcb['Gói cước'] = np.where(map_goicuoc.isna(),' ',map_goicuoc)
          # tmp_tcb['Gói cước'] = tmp_tcb['Gói cước'].replace(['', ' ', 'nan'], np.nan)
          tmp_tcb.loc[(tmp_tcb['Gói cước'].astype(str).str.strip().isin(['', ' ', 'nan'])) & (tmp_tcb['Loại đơn hàng'] == 'VOUCHER_DATA'),'Gói cước'] = 'VOUCHER_DATA'


          tmp_tcb['Loại sim'] = np.where(tmp_tcb['Loại đơn hàng'] == 'Mua sim',tmp_tcb['Mã đơn hàng'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Loại sim']).fillna('0'),
                              np.where(tmp_tcb['Loại đơn hàng'] == 'Đổi sim',tmp_tcb['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Loại sim']).fillna('0'),'0'))

          tmp_tcb_affiliate = tmp_tcb['Mã đơn hàng'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Affiliate partner'])
          tmp_tcb['Đơn vị'] = np.where(tmp_tcb_affiliate == 'ACCESSTRADE','SALES - Online - ACT','SALES - Online')
          # tmp_tcb['Co'] = tmp_tcb['Giá trị giao dịch']

          so_luong_sim = dwh_bh['Mã vận đơn MBC'].value_counts()
          tmp_tcb['Số lượng sim'] = tmp_tcb['Mã đơn hàng'].map(so_luong_sim).fillna(0).astype(int)

          # 1. Chọn các dòng có Phí hòa mạng > 0
          mask_hoa_mang = dwh_bh['Giá hòa mạng'] > 0

          # 2. Đếm số lần mỗi Mã vận đơn MBC phát sinh Phí hòa mạng
          so_luong_hoa_mang = (dwh_bh.loc[mask_hoa_mang, 'Mã vận đơn MBC'].value_counts())

          # 3. Map vào tmp_tcb_MISA
          tmp_tcb['Số lượng hòa mạng'] = (tmp_tcb['Mã đơn hàng'].map(so_luong_hoa_mang).fillna(0).astype(int))


          tong_gia_sim = dwh_bh.groupby('Mã vận đơn MBC')['Giá sim'].sum()
          tmp_tcb['Tiền sim trắng'] = np.where(tmp_tcb['Loại đơn hàng'] == "Đổi sim",5000,
                                              np.where(tmp_tcb['Loại đơn hàng'] == "Mua sim",tmp_tcb['Mã đơn hàng'].map(tong_gia_sim).fillna(0),0))

          tong_gia_hoa_mang = dwh_bh.groupby('Mã vận đơn MBC')['Giá hòa mạng'].sum()
          tmp_tcb['Phí hòa mạng'] = np.where(tmp_tcb['Loại đơn hàng'] == "Mua sim",tmp_tcb['Mã đơn hàng'].map(tong_gia_hoa_mang).fillna(0),0)

          tong_gia_thoai = dwh_bh.groupby('Mã vận đơn MBC')['Giá thoại2'].sum()
          tmp_tcb['Tiền gói - Thoại'] = np.where(tmp_tcb['Loại đơn hàng'] == "Mua gói cước", tmp_tcb['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Thoại']).fillna(0).astype(int),
                                      np.where(tmp_tcb['Loại đơn hàng'] == "Mua sim",tmp_tcb['Mã đơn hàng'].map(tong_gia_thoai).fillna(0),0))

          tong_gia_sms = dwh_bh.groupby('Mã vận đơn MBC')['Giá SMS2'].sum()
          tmp_tcb['Tiền gói - SMS'] = np.where(tmp_tcb['Loại đơn hàng'] == "Mua gói cước", tmp_tcb['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá SMS2']).fillna(0).astype(int),
                                      np.where(tmp_tcb['Loại đơn hàng'] == "Mua sim",tmp_tcb['Mã đơn hàng'].map(tong_gia_sms).fillna(0),0))

          tong_gia_data = dwh_bh.groupby('Mã vận đơn MBC')['Giá data2'].sum()

          tmp_tcb['Tiền gói - Data'] = np.where(tmp_tcb['Loại đơn hàng'] == "Mua gói cước", tmp_tcb['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Data2']).fillna(0).astype(int),
                                      np.where(tmp_tcb['Loại đơn hàng'] == "Mua sim",tmp_tcb['Mã đơn hàng'].map(tong_gia_data).fillna(0),
                                      np.where(tmp_tcb['Loại đơn hàng'] == "VOUCHER_DATA",tmp_tcb['Tổng tiền thu'].fillna(0),0)))


          tong_gia_combo = dwh_bh.groupby('Mã vận đơn MBC')['Combo - 1 tháng'].sum()

          tmp_tcb['Tiền gói - Combo 1 tháng'] = np.where(tmp_tcb['Loại đơn hàng'].isin(["Mua gói cước", "Gia hạn gói cước"]),tmp_tcb['Mã đơn hàng'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá combo']).fillna(0).astype(int),
                                      np.where(tmp_tcb['Loại đơn hàng'] == "Mua sim",tmp_tcb['Mã đơn hàng'].map(tong_gia_combo).fillna(0),0))


          tmp_tcb['Doanh thu khác'] = np.where(tmp_tcb['Loại đơn hàng'] == "Đổi sim",20000,np.where(tmp_tcb['Loại đơn hàng'] == "Khôi phục số",tmp_tcb['Tổng tiền thu'],0))

          tmp_tcb['Tiền ship'] = np.where(tmp_tcb['Loại đơn hàng'] == "Đổi sim",tmp_tcb['Tổng tiền thu'] - tmp_tcb['Tiền sim trắng'] - tmp_tcb['Doanh thu khác'],
                                      np.where(tmp_tcb['Loại đơn hàng'] == "Mua sim",tmp_tcb['Mã đơn hàng'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').drop_duplicates('Mã đơn hàng').set_index('Mã vận đơn MBC')['Phí ship']).fillna(0),0))

          tmp_tcb['Topup'] = np.where(tmp_tcb['Loại đơn hàng'] == "Topup",tmp_tcb['Tổng tiền thu'],0)

          tmp_tcb['Voucher'] = np.where(tmp_tcb['Loại đơn hàng'] == "VOUCHER",tmp_tcb['Tổng tiền thu'],0)
          cols_to_subtract = ['Tiền sim trắng', 'Phí hòa mạng', 'Tiền gói - Thoại', 'Tiền gói - SMS','Tiền gói - Combo 1 tháng'  ,  'Tiền gói - Data',  'Doanh thu khác','Tiền ship', 'Topup', 'Voucher']
          tmp_tcb['Check'] = tmp_tcb['Tổng tiền thu'] - tmp_tcb[cols_to_subtract].sum(axis=1)
          TCB_list.append(tmp_tcb)

        else:
          print(f"⚠️ File CSV {filename} không đủ cột.")
      else:
        print(f"⚠️ File CSV {filename} không đủ dòng.")

  except Exception as e: # Bắt lỗi
    print(f"❌ Lỗi xử lý file: {filename}") # File bị lỗi
    print(f"Chi tiết lỗi: {e}") # Tên Lỗi
# Concat data các cổng thanh toán
if BIDV_list:
    BIDV = pd.concat(BIDV_list, ignore_index=True)
    BIDV = BIDV.drop_duplicates(keep='first')

if ONEPAY_list:
    ONE_PAY = pd.concat(ONEPAY_list, ignore_index=True)
    ONE_PAY = ONE_PAY.drop_duplicates(keep='first')

if VNPAY_list:
    EPAY = pd.concat(VNPAY_list, ignore_index=True)
    EPAY = EPAY.drop_duplicates(keep='first')

if TCB_list:
    TCB = pd.concat(TCB_list, ignore_index=True)
    TCB = TCB.drop_duplicates(keep='first')



In [ ]:
print(TCB.shape)
print(EPAY.shape)
print(ONE_PAY.shape)
print(BIDV.shape)


(741, 28)
(201, 53)
(1814, 40)
(5259, 25)


In [ ]:
tmp_onepay['Gói cước'].unique()

array([' ', 'WIN119P', 'D15', 'D5', 'WIN69X', 'WIN89X', 'WIN89S', 'WIN60',
       'TỰ DO TRẢI NGHIỆM', 'D35X2-69', 'D30', 'V10', 'V30', 'WINLITE',
       'D50', 'CK149', 'V20', 'WIN159P'], dtype=object)

In [ ]:
# process tách df Reconcile và df cần check lại

In [ ]:
EPAY_REC = EPAY.copy()
EPAY_REC = EPAY_REC[~(EPAY_REC['Check'] != 0)]
EPAY_nan = EPAY.copy()
EPAY_nan = EPAY_nan[(EPAY_nan['Check']!= 0)]
EPAY_nan = EPAY_nan.rename(columns={'Mã đặt hàng' : 'REF','Ngày giao dịch' : 'Date'})

In [ ]:
ONE_PAY_REC = ONE_PAY.copy()
ONE_PAY_REC = ONE_PAY_REC[~(ONE_PAY_REC['Check'] != 0)]
ONE_PAY_nan = ONE_PAY.copy()
ONE_PAY_nan = ONE_PAY_nan[(ONE_PAY_nan['Check']!= 0)]
ONE_PAY_nan = ONE_PAY_nan.rename(columns={'Mã đơn hàng' : 'REF','Ngày giao dịch' : 'Date'})

In [ ]:
TCB_REC = TCB.copy()
TCB_REC = TCB_REC[~(TCB_REC['Check'] != 0)]
TCB_nan = TCB.copy()
TCB_nan = TCB_nan[(TCB_nan['Check']!= 0)]
TCB_nan = TCB_nan.rename(columns={'So but toan/Txn No' : 'REF','Ngay hach toan/Booking Date' : 'Date'})

In [ ]:

BIDV_REC = BIDV.copy()
BIDV_REC = BIDV_REC[~(BIDV_REC['Loại đơn hàng'] =="Không có trong DWH")]
BIDV_nan = BIDV.copy()
BIDV_nan = BIDV_nan[(BIDV_nan['Loại đơn hàng'] =="Không có trong DWH")]
BIDV_nan = BIDV_nan.rename(columns={'Mã Hóa đơn' : 'REF','Ngày hạch toán' : 'Date','Số tiền thu hộ' : 'Topup'})


In [ ]:
#lấy columns của df cần check lại với IT
columns_nan = [ 'Date', 'REF','SDT','KYC', 'Tổng tiền thu', 'Loại đơn hàng', 'Gói cước', 'Loại sim', 'Đơn vị', 'Số lượng sim', 'Số lượng hòa mạng', 'Tiền sim trắng',
               'Phí hòa mạng', 'Tiền gói - Thoại', 'Tiền gói - SMS', 'Tiền gói - Data', 'Tiền gói - Combo 1 tháng', 'Doanh thu khác', 'Tiền ship', 'Topup', 'Voucher','Check','Giảm giá','Source']

In [ ]:
EPAY_nan['Source'] = 'EPAY'
ONE_PAY_nan['Source'] = 'ONEPAY'
TCB_nan['Source'] = 'TCB'
BIDV_nan['Source'] = 'BIDV'

EPAY_nan = EPAY_nan[columns_nan]
ONE_PAY_nan = ONE_PAY_nan[columns_nan]
TCB_nan = TCB_nan[columns_nan]
BIDV_nan = BIDV_nan[['Date','REF','SDT','KYC','Loại đơn hàng','Topup','Source']]
DCHECK = pd.concat([EPAY_nan,ONE_PAY_nan,TCB_nan,BIDV_nan], ignore_index=True)


In [ ]:
# lấy columns của df đã reconcile
columns_misa = [ 'Date','SDT','KYC', 'Tổng tiền thu', 'Loại đơn hàng', 'Gói cước', 'Loại sim', 'Đơn vị', 'Số lượng sim', 'Số lượng hòa mạng',
 'Tiền sim trắng', 'Phí hòa mạng', 'Tiền gói - Thoại', 'Tiền gói - SMS', 'Tiền gói - Data', 'Tiền gói - Combo 1 tháng', 'Doanh thu khác', 'Tiền ship', 'Topup', 'Voucher','Source']

In [ ]:
## Xử lý gộp df đã reconcile

In [ ]:
TCB_MISA = TCB_REC.copy()
TCB_MISA['KYC'] = TCB_MISA['KYC'].fillna(1)
TCB_MISA['Source'] = 'TCB'
TCB_MISA = TCB_MISA.rename(columns = {'Ngay hach toan/Booking Date' : 'Date'})
TCB_MISA = TCB_MISA[columns_misa]


In [ ]:
TCB_MISA_group = TCB_MISA.groupby(['Date', 'Gói cước','Đơn vị']).sum(numeric_only=True).reset_index()
TCB_MISA_group['Số lượng'] = TCB_MISA.groupby(['Date', 'Gói cước','Đơn vị']).size().values


In [ ]:
ONE_PAY_MISA = ONE_PAY_REC.copy()
ONE_PAY_MISA['KYC'] = ONE_PAY_MISA['KYC'].fillna(1)
ONE_PAY_MISA['Source'] = "ONEPAY"
ONE_PAY_MISA = ONE_PAY_MISA.rename(columns = {'Ngày giao dịch' : 'Date'})

In [ ]:
ONE_PAY_MISA = ONE_PAY_MISA[columns_misa]
ONE_PAY_MISA_group = ONE_PAY_MISA.groupby(['Date', 'Gói cước','Đơn vị']).sum(numeric_only=True).reset_index()
ONE_PAY_MISA_group['Số lượng'] = ONE_PAY_MISA.groupby(['Date', 'Gói cước','Đơn vị']).size().values


In [ ]:
ONE_PAY_MISA_group['Tổng tiền thu'].sum()

np.int64(129852500)

In [ ]:
EPAY_MISA = EPAY_REC.copy()
EPAY_MISA['KYC'] = EPAY_MISA['KYC'].fillna(1)
EPAY_MISA['Source'] = 'EPAY'
EPAY_MISA = EPAY_MISA.rename(columns = {'Ngày giao dịch' : 'Date'})

In [ ]:
EPAY_MISA = EPAY_MISA[columns_misa]
EPAY_MISA_group = EPAY_MISA.groupby(['Date', 'Gói cước','Đơn vị']).sum(numeric_only=True).reset_index()
EPAY_MISA_group['Số lượng'] = EPAY_MISA.groupby(['Date', 'Gói cước','Đơn vị']).size().values


In [ ]:
BIDV_MISA = BIDV_REC.copy()
BIDV_MISA['KYC'] = BIDV_MISA['KYC'].fillna(1)
BIDV_MISA['Source'] = "BIDV"
BIDV_MISA = BIDV_MISA.rename(columns = {'Ngày hạch toán' : 'Date','Số tiền thu hộ' : 'Topup'})
BIDV_MISA['Tổng tiền thu'] = BIDV_MISA['Topup']
BIDV_MISA = BIDV_MISA[['Date','SDT','KYC','Loại đơn hàng','Topup', 'Tổng tiền thu','Đơn vị',"Source"]]
BIDV_MISA['Topup'].sum()

np.int64(368171939)

In [ ]:
BIDV_MISA_group = (
    BIDV_MISA
    .groupby(['Date','Đơn vị'], as_index=False)
    .sum(numeric_only=True)  # Chỉ tính tổng các cột số
)


In [ ]:
# Tính Số lượng sim và hòa mạng
df_sim = pd.concat([TCB_MISA,EPAY_MISA,ONE_PAY_MISA,BIDV_MISA], ignore_index=True)
df_sim = df_sim[df_sim['Loại sim'].isin(['Esim', 'Sim vật lý', 'E sim'])]
df_sim = df_sim.copy()
df_sim["Loại sim"] = df_sim["Loại sim"].str.replace("E sim", "Esim")

df_sim = df_sim[['Date','SDT','Loại sim', 'Đơn vị', 'Số lượng sim', 'Số lượng hòa mạng', 'Tiền sim trắng','Source']]

In [ ]:
df_sim_group = (
    df_sim
    .groupby(['Date', 'Loại sim','Source','Đơn vị'], as_index=False)
    .sum(numeric_only=True)  # Chỉ tính tổng các cột số
)

In [ ]:
Sim_melt = pd.melt(
    df_sim_group,
    id_vars=['Date', 'Loại sim', 'Đơn vị','Source'],  # giữ nguyên các cột này
    value_vars=["Số lượng sim","Số lượng hòa mạng", "Tiền sim trắng"],
    var_name='Type',   # tên cột mới cho loại
    value_name='Amount'         # tên cột mới cho giá trị
)
Sim_melt = Sim_melt[Sim_melt['Amount'] != 0]

Sim_melt['Non-VAT'] = np.where(
    Sim_melt['Type'].isin(["Số lượng hòa mạng","Số lượng sim"]),
    np.nan,
    Sim_melt['Amount'] / 1.1
)
Sim_melt['VAT'] = np.where(
    Sim_melt['Type'].isin(["Số lượng hòa mạng","Số lượng sim"]),
    np.nan,
    Sim_melt['Amount'] - Sim_melt['Non-VAT']
)

In [ ]:
Sim_melt["Tài khoản"] = np.where(
    Sim_melt["Type"] == "Tiền sim trắng",
    "338711",
    ""
)


In [ ]:
TCB_MISA_group['Source'] = 'TCB'
EPAY_MISA_group['Source'] = 'EPAY'
ONE_PAY_MISA_group['Source'] = 'ONEPAY'
BIDV_MISA_group['Source'] = 'BIDV'
MISA = pd.concat([TCB_MISA_group,EPAY_MISA_group,ONE_PAY_MISA_group,BIDV_MISA_group], ignore_index=True)
MISA['Gói cước'] = MISA['Gói cước'].replace('TỰ DO TRẢI NGHIỆM', 'TU DO TRAI NGHIEM')
print(MISA.shape)



(167, 19)


In [ ]:
EPAY_MISA

2,Date,SDT,KYC,Tổng tiền thu,Loại đơn hàng,Gói cước,Loại sim,Đơn vị,Số lượng sim,Số lượng hòa mạng,...,Phí hòa mạng,Tiền gói - Thoại,Tiền gói - SMS,Tiền gói - Data,Tiền gói - Combo 1 tháng,Doanh thu khác,Tiền ship,Topup,Voucher,Source
0,23-06-2025,0559302798,1.0,69000,Mua gói cước,WIN69X,0,SALES - Online,0,0,...,0.0,0.0,0.0,0.0,69000.0,0,0.0,0,0,EPAY
1,23-06-2025,0559521057,1.0,20000,Topup,,0,SALES - Online,0,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,20000,0,EPAY
2,23-06-2025,0559526292,1.0,100000,Topup,,0,SALES - Online,0,0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,100000,0,EPAY
3,23-06-2025,0559896311,1.0,35000,Mua gói cước,WIN69X,0,SALES - Online,0,0,...,0.0,0.0,0.0,0.0,35000.0,0,0.0,0,0,EPAY
4,23-06-2025,0559339061,1.0,55000,Mua gói cước,WIN69X,0,SALES - Online,0,0,...,0.0,0.0,0.0,0.0,55000.0,0,0.0,0,0,EPAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,21-06-2025,0559059427,1.0,69000,Mua gói cước,WIN69X,0,SALES - Online,0,0,...,0.0,0.0,0.0,0.0,69000.0,0,0.0,0,0,EPAY
197,21-06-2025,0559355920,1.0,119000,Mua sim,WIN69X,Esim,SALES - Online,1,1,...,25000.0,0.0,0.0,0.0,69000.0,0,0.0,0,0,EPAY
198,21-06-2025,0559189605,1.0,5000,Mua gói cước,D5,0,SALES - Online,0,0,...,0.0,0.0,0.0,5000.0,0.0,0,0.0,0,0,EPAY
199,21-06-2025,0559607015,1.0,69000,Mua gói cước,WIN69X,0,SALES - Online,0,0,...,0.0,0.0,0.0,0.0,69000.0,0,0.0,0,0,EPAY


In [ ]:
EPAY_MISA['Tổng tiền thu'].sum()

np.int64(12369000)

In [ ]:
tong_tien_epay = MISA.loc[MISA['Source'] == 'EPAY', 'Tổng tiền thu'].sum()
print(f"Tổng tiền thu từ EPAY là: {tong_tien_epay:,}")


Tổng tiền thu từ EPAY là: 12,369,000


In [ ]:
# check số trước và sau khi gộp
MISA['Tổng tiền thu'].sum()-(MISA[['Tiền sim trắng',    'Phí hòa mạng',    'Tiền gói - Thoại',    'Tiền gói - SMS',
    'Tiền gói - Data',    'Tiền gói - Combo 1 tháng',    'Doanh thu khác',    'Tiền ship',    'Topup','Voucher']].sum().sum())

np.float64(0.0)

In [ ]:
# các cột cần gộp giá trị
value_vars = [
    'Tổng tiền thu',
    'Số lượng sim',
    'Số lượng hòa mạng',
    'Tiền sim trắng',
    'Phí hòa mạng',
    'Tiền gói - Thoại',
    'Tiền gói - SMS',
    'Tiền gói - Data',
    'Tiền gói - Combo 1 tháng',
    'Doanh thu khác',
    'Tiền ship',
    'Topup',
    'Voucher',
]


MISA_melt = pd.melt(
    MISA,
    id_vars=['Date', 'Gói cước', 'Source','Đơn vị','Số lượng'],  # giữ nguyên các cột này
    value_vars=value_vars,
    var_name='Type',   # tên cột mới cho loại
    value_name='Amount'         # tên cột mới cho giá trị
)


In [ ]:
loai_can_dem = ['Tiền gói - Thoại', 'Tiền gói - SMS', 'Tiền gói - Data','Tiền gói - Combo 1 tháng']
# Gán lại Số_lượng_giao_dịch: chỉ giữ giá trị nếu Type thuộc nhóm

# Ép về str để xử lý đồng bộ
MISA_melt['Gói cước'] = MISA_melt['Gói cước'].astype(str)

# Clear các giá trị không hợp lệ về NaN
MISA_melt.loc[
    (MISA_melt['Gói cước'].str.lower() == 'nan') |
    (MISA_melt['Gói cước'] == '')|
    (MISA_melt['Gói cước'] == ' '),
    'Gói cước'
] = np.nan

MISA_melt['Số lượng'] = np.where(
    (MISA_melt['Type'].isin(loai_can_dem)) &
    (MISA_melt['Gói cước'].notna()),
    MISA_melt['Số lượng'],
    np.nan
)
# loai_goi = ['Tiền gói - Thoại', 'Tiền gói - SMS', 'Tiền gói - Data', 'Tiền gói - Combo 1 tháng']

MISA_melt['Gói cước'] = MISA_melt.apply(
    lambda row: row['Gói cước'] if row['Type'] in loai_can_dem else "",
    axis=1
)


In [ ]:
MISA_melt = MISA_melt[(MISA_melt['Amount'].notna()) &(MISA_melt['Amount'] != 0)]
MISA_melt = MISA_melt.copy()
MISA_melt['Số lượng'] = MISA_melt['Số lượng'].astype(float)


# MISA_melt["Type"].unique()

In [ ]:
MISA_melt[MISA_melt['Type'].isin(['Số lượng hòa mạng','Tiền sim trắng', 'Phí hòa mạng', 'Tiền gói - Thoại','Tiền gói - SMS', 'Tiền gói - Data', 'Tiền gói - Combo 1 tháng','Doanh thu khác', 'Tiền ship', 'Topup', 'Voucher','Số_lượng_giao_dịch'])]['Amount'].sum()

np.float64(558032647.0)

In [ ]:
# Convert sang tài khoản
mapping_dict = {
    'Phí hòa mạng': '338712',
    'Tiền gói - Thoại': '338721',
    'Tiền gói - SMS': '338722',
    'Tiền gói - Data': '338723',
    'Tiền ship': '33875',
    'Topup': '3387311',
    'Doanh thu khác': '51185',
    'Voucher': '51113',
    'Tiền gói - Combo 1 tháng': '3387241',
}


In [ ]:
# Loại những loại k đưa vào MISA
MISA_melt = MISA_melt[~MISA_melt['Type'].isin(["Tổng tiền thu","Số lượng hòa mạng","Số lượng sim","Tiền sim trắng"])].copy()
# Map tài khoản
MISA_melt['Tài khoản'] = MISA_melt['Type'].map(mapping_dict)
MISA_melt['Số lượng'] = MISA_melt['Số lượng'].fillna(1)


In [ ]:
MISA_melt[MISA_melt['Type'] == 'Tiền gói - Data'].sort_values(by='Source')

,Date,Gói cước,Source,Đơn vị,Số lượng,Type,Amount,Tài khoản
1255,23-06-2025,D5,EPAY,SALES - Online,5.0,Tiền gói - Data,25000.0,338723
1257,23-06-2025,TU DO TRAI NGHIEM,EPAY,SALES - Online,5.0,Tiền gói - Data,225000.0,338723
1256,23-06-2025,D50,EPAY,SALES - Online,1.0,Tiền gói - Data,50000.0,338723
1254,23-06-2025,D35X2-69,EPAY,SALES - Online,1.0,Tiền gói - Data,35000.0,338723
1253,23-06-2025,D15,EPAY,SALES - Online,1.0,Tiền gói - Data,15000.0,338723
1246,22-06-2025,VOUCHER_DATA,EPAY,SALES - Online,1.0,Tiền gói - Data,5000.0,338723
1243,22-06-2025,TU DO TRAI NGHIEM,EPAY,SALES - Online,5.0,Tiền gói - Data,270000.0,338723
1242,22-06-2025,D50,EPAY,SALES - Online,1.0,Tiền gói - Data,50000.0,338723
1241,22-06-2025,D5,EPAY,SALES - Online,8.0,Tiền gói - Data,40000.0,338723
1240,22-06-2025,D35X2-69,EPAY,SALES - Online,1.0,Tiền gói - Data,35000.0,338723


In [ ]:
# Gộp lại theo ngày, gói, tài khoản, TYpe
MISA_melt2 =  (
    MISA_melt
    .groupby(['Date', 'Gói cước','Source','Đơn vị','Type','Tài khoản'],dropna = False, as_index=False)
    .sum(numeric_only=True)  # Chỉ tính tổng các cột số
)

In [ ]:
MISA_melt2[MISA_melt2['Type'] == 'Voucher'].sort_values(by='Source')

,Date,Gói cước,Source,Đơn vị,Type,Tài khoản,Số lượng,Amount
142,23-06-2025,,EPAY,SALES - Online,Voucher,51113,1.0,85000.0
50,21-06-2025,,ONEPAY,SALES - Online,Voucher,51113,1.0,8585000.0
107,22-06-2025,,ONEPAY,SALES - Online,Voucher,51113,1.0,552500.0
149,23-06-2025,,ONEPAY,SALES - Online,Voucher,51113,1.0,11900000.0
151,23-06-2025,,TCB,SALES - Online,Voucher,51113,1.0,18000.0


In [ ]:
# loại số lượng count không đúng
MISA_melt2['Số lượng'] = np.where(
    (MISA_melt2['Gói cước'] == "TỰ DO TRẢI NGHIỆM") & (MISA_melt2['Type'] != "Tiền gói - Data"),
    np.nan,
    MISA_melt2['Số lượng']
)

MISA_melt2['Số lượng'] = MISA_melt2['Số lượng'].replace(0, np.nan)


In [ ]:
#Tính VAT và Non-VAT
MISA_melt2['Non-VAT'] = np.where(MISA_melt2['Type'] == "Voucher", MISA_melt2['Amount'],
                                  MISA_melt2['Amount']/1.1)
MISA_melt2['VAT'] = MISA_melt2['Amount'] - MISA_melt2['Non-VAT']
MISA_melt2['VAT'] = np.where(MISA_melt2['Type'] == 'Voucher', np.nan, MISA_melt2['VAT'])


In [ ]:
MISA_melt2[MISA_melt2['Type'] == 'Voucher']

,Date,Gói cước,Source,Đơn vị,Type,Tài khoản,Số lượng,Amount,Non-VAT,VAT
50,21-06-2025,,ONEPAY,SALES - Online,Voucher,51113,1.0,8585000.0,8585000.0,NaN
107,22-06-2025,,ONEPAY,SALES - Online,Voucher,51113,1.0,552500.0,552500.0,NaN
142,23-06-2025,,EPAY,SALES - Online,Voucher,51113,1.0,85000.0,85000.0,NaN
149,23-06-2025,,ONEPAY,SALES - Online,Voucher,51113,1.0,11900000.0,11900000.0,NaN
151,23-06-2025,,TCB,SALES - Online,Voucher,51113,1.0,18000.0,18000.0,NaN


In [ ]:
MISA_Excel = pd.concat([MISA_melt2,Sim_melt], ignore_index=True)

In [ ]:
#Generate mã chứng từ
MISA_Excel['Số chứng từ'] = (
    MISA_Excel['Source'] +
    MISA_Excel['Date'].str.split('-').str[2].str[-2:] +
    MISA_Excel['Date'].str.split('-').str[1] +
    MISA_Excel['Date'].str.split('-').str[0]
)

# Tạo STT bằng rank (kiểu int)
MISA_Excel['STT'] = np.where(
    MISA_Excel['Type'] == 'Voucher',
    2,
    1
)


# Ghép lại
MISA_Excel['Số chứng từ'] = (
    MISA_Excel['Số chứng từ'] + "-" +
    MISA_Excel['STT'].astype(str)
)



In [ ]:
# Generate diễn giải hạch toán
MISA_Excel['Diễn giải'] = 'Bán hàng Khách hàng không lấy hóa đơn - TT qua ' + MISA_Excel['Source'] + ' ngày ' + MISA_Excel['Date'] + '-' + MISA_Excel['Type']
MISA_Excel['Mã KH'] = MISA_Excel['Source'] + '-KL'
MH_mapping = {
    'Topup': 'DVVT',
    'Phí hòa mạng': 'HM',
    'Doanh thu khác': 'DV',
    'Tiền ship': 'VC',
    'Tiền sim trắng': 'ST',
    'Voucher': 'VCPL',
}

# Danh sách các Type sẽ lấy theo Gói cước
goi_cuoc_types = [
    'Tiền gói - Thoại',
    'Tiền gói - SMS',
    'Tiền gói - Data',
    'Tiền gói - Combo 1 tháng'
]

def get_ma_hang(row):
    if row['Type'] in MH_mapping:
        return MH_mapping[row['Type']]
    elif row['Type'] in goi_cuoc_types:
        return row['Gói cước']
    else:
        return ''

MISA_Excel['Mã hàng'] = MISA_Excel.apply(get_ma_hang, axis=1)


In [ ]:
# - Lấy template input Misa
template = '/content/drive/My Drive/KeToan_HachToanGiaoDich/template/TemplateMISA.xls'
df_template = pd.read_excel(template)

In [ ]:
#Thao tác cho file input misa
df_template['Hiển thị trên sổ'] = '0'
df_template['Kiêm phiếu xuất kho'] = '0'
df_template['Ngày chứng từ (*)'] = MISA_Excel['Date']
df_template['TK Tiền/Chi phí/Nợ (*)'] = '1311'
df_template['TK Doanh thu/Có (*)'] = MISA_Excel['Tài khoản']
df_template['% thuế GTGT'] = '10'
df_template['Thành tiền'] = MISA_Excel['Non-VAT']
df_template['Tiền thuế GTGT'] = MISA_Excel['VAT']
df_template['TK thuế GTGT'] = '33311'
df_template['Đơn vị'] = MISA_Excel['Đơn vị']
df_template['Mã thống kê'] = "N"
df_template['Số chứng từ (*)'] = MISA_Excel['Số chứng từ']
df_template['Mã hàng (*)'] = MISA_Excel['Mã hàng']
df_template['Diễn giải'] = MISA_Excel['Diễn giải']
df_template['Mã khách hàng'] = MISA_Excel['Mã KH']
df_template['% thuế GTGT'] = np.where(df_template['Mã hàng (*)'] == 'VCPL',np.nan,df_template['% thuế GTGT'])
df_template['TK thuế GTGT'] = np.where(df_template['Mã hàng (*)'] == 'VCPL',np.nan,df_template['TK thuế GTGT'])

df_template = df_template[~df_template['Diễn giải'].str.contains('Số lượng')]

In [ ]:
df_template.info()

<class 'pandas.core.frame.DataFrame'>
Index: 217 entries, 0 to 242
Data columns (total 57 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Hiển thị trên sổ                                    0 non-null      object 
 1   Hình thức bán hàng                                  0 non-null      object 
 2   Phương thức thanh toán                              0 non-null      object 
 3   Kiêm phiếu xuất kho                                 0 non-null      object 
 4   XK vào khu phi thuế quan và các TH được coi như XK  0 non-null      object 
 5   Lập kèm hóa đơn                                     0 non-null      object 
 6   Đã lập hóa đơn                                      0 non-null      object 
 7   Ngày hạch toán (*)                                  0 non-null      object 
 8   Ngày chứng từ (*)                                   217 non-null    object 
 9   Số c

In [ ]:
# Xuất file input Misa
df_template.to_excel(f'/content/drive/My Drive/KeToan_HachToanGiaoDich/File_output/Misa_CTT_upload_{timestamp}.xlsx', index=False)


In [ ]:
Summary = MISA_Excel.copy()

In [ ]:
Summary = Summary.iloc[:, :12]


In [ ]:
# Tạo File Excel tổng hợp từ báo cáo và CTT để check lại số liệu
with pd.ExcelWriter(f'/content/drive/My Drive/KeToan_HachToanGiaoDich/File_output/Doi Soat CTT - {timestamp}.xlsx', engine='xlsxwriter') as writer:
    EPAY.to_excel(writer, sheet_name='Epay', index=False)
    ONE_PAY.to_excel(writer, sheet_name='ONE_PAY', index=False)
    BIDV.to_excel(writer, sheet_name='BIDV', index=False)
    TCB.to_excel(writer, sheet_name='TCB', index=False)
    dwh.to_excel(writer, sheet_name='BC Cổng thanh toán', index=False)
    dwh_bh.to_excel(writer, sheet_name='BC Tổng hợp bán hàng', index=False)
    DCHECK.to_excel(writer, sheet_name='Check lại', index=False)
    Summary.to_excel(writer, sheet_name='Tổng hợp', index=False)

In [ ]:
# Tính thời gian chạy
elapsed_time = time.time() - start_time
print(f"Thời gian chạy: {elapsed_time:.2f} giây")

Thời gian chạy: 1042.09 giây
